# `BaikalETK`: NRPy+-Powered BSSN Solver for the Einstein Toolkit

## Author: Zach Etienne

## This module generates `BaikalETK`, an [Einstein Toolkit](https://einsteintoolkit.org) thorn for solving Einstein's equations in the BSSN formalism, in Cartesian coordinates. It features SIMD intrinsics and OpenMP support.

**Module Status:** <font color='red'><b> Not yet validated, work-in-progress </b></font>

**Validation Notes:** This tutorial module has not yet been validated.

## Introduction

[Lake Baikal](https://en.wikipedia.org/wiki/Lake_Baikal) is home to the [nerpa seal](https://en.wikipedia.org/wiki/Baikal_seal), NRPy+'s mascot.

```
How often did my soul cry out:
Come back to Baikal once again?
I still do not know this lake:
To see does not mean to know.
```
[Igor Severyanin](https://en.wikipedia.org/wiki/Igor_Severyanin), [[1]](https://1baikal.ru/en/istoriya/let’s-turn-to-baikal-a-poetic-view).

This thorn is meant to reproduce the functionality of the `McLachlan` thorn, generated by the [Mathematica](https://www.wolfram.com/mathematica/)-based [Kranc](http://kranccode.org/) code, but using the NRPy+ infrastructure.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#initializenrpy): Initialize needed Python/NRPy+ modules
1. [Step 2](#bssn): Output C code for BSSN spacetime solve
    1. [Step 2.a](#bssnrhs): BSSN RHS expressions
    1. [Step 2.b](#hammomconstraints): Hamiltonian & momentum constraints
    1. [Step 2.c](#gamconstraint): Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (in Cartesian coordinates, $\det{\hat{\gamma}_{ij}}=1$)
    1. [Step 2.d](#parallel_codegen): Generate all C codes in parallel
1. [Step 3](#cclfiles): CCL files - Define how this module interacts and interfaces with the wider Einstein Toolkit infrastructure
    1. [Step 3.a](#paramccl): `param.ccl`: specify free parameters within `BaikalETK`
1. [Step N-1](#code_validation): Code Validation
1. [Step N](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='initializenrpy'></a>

# Step 1: Initialize needed Python/NRPy+ modules \[Back to [top](#toc)\]

$$\label{initializenrpy}$$

In [1]:
# Step 1: Import needed core NRPy+ modules
from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions

# FIXME: NOT CROSS-PLATFORM:
!rm -rf BaikalETK

# Create directory for BaikalETK thorn & subdirectories
outrootdir = "BaikalETK/"
cmd.mkdir(os.path.join(outrootdir))
outdir = os.path.join(outrootdir,"src") # Main C code output directory
cmd.mkdir(outdir)

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 2: Set some core parameters, including CoordSystem MoL timestepping algorithm,
#                                 FD order, floating point precision, and CFL factor:
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical, 
#              SymTP, SinhSymTP
# NOTE: Only CoordSystem == Cartesian makes sense here; new 
#       boundary conditions are needed within the ETK for 
#       Spherical, etc. coordinates.
CoordSystem     = "Cartesian"

par.set_parval_from_str("reference_metric::CoordSystem",CoordSystem)
rfm.reference_metric() # Create ReU, ReDD needed for rescaling B-L initial data, generating BSSN RHSs, etc.

LapseCondition  = "OnePlusLog"
# Set the standard, second-order advecting-shift, Gamma-driving shift condition
ShiftCondition  = "GammaDriving2ndOrder_NoCovariant"

FD_order  = 6           # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "CCTK_REAL" # Set REAL to CCTK_REAL, the ETK data type for 
                        # floating point precision (typically `double`)
# Set finite differencing order:
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# Copy SIMD/SIMD_intrinsics.h to $outdir/SIMD/SIMD_intrinsics.h
cmd.mkdir(os.path.join(outdir,"SIMD"))
shutil.copy(os.path.join("SIMD/")+"SIMD_intrinsics.h",os.path.join(outdir,"SIMD/"))

# Set the gridfunction memory access type to ETK-like, so that finite_difference
#    knows how to read and write gridfunctions from/to memory.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

<a id='bssn'></a>

# Step 2: Output C code for BSSN spacetime solve \[Back to [top](#toc)\]
$$\label{bssn}$$

<a id='bssnrhs'></a>

## Step 2.a: BSSN RHS expressions \[Back to [top](#toc)\]
$$\label{bssnrhs}$$

In [2]:
import time # Standard Python module; useful for benchmarking below expression & code generation.

import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as gaugerhs
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", ShiftCondition)
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::LapseEvolutionOption", LapseCondition)

print("Generating symbolic expressions for BSSN RHSs...")
start = time.time()
# Enable rfm_precompute infrastructure, which results in 
#   BSSN RHSs that are free of transcendental functions,
#   even in curvilinear coordinates, so long as 
#   ConformalFactor is set to "W" (default).
cmd.mkdir(os.path.join(outdir,"rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute","True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir",os.path.join(outdir,"rfm_files/"))

# Evaluate BSSN + BSSN gauge RHSs with rfm_precompute enabled:
import BSSN.BSSN_quantities as Bq
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","True")

rhs.BSSN_RHSs()
gaugerhs.BSSN_gauge_RHSs()

# Add Kreiss-Oliger dissipation to the BSSN RHSs:
thismodule = "KO_Dissipation"
diss_strength = par.Cparameters("REAL", thismodule, "diss_strength", 0.1)

alpha_dKOD = ixp.declarerank1("alpha_dKOD")
cf_dKOD    = ixp.declarerank1("cf_dKOD")
trK_dKOD   = ixp.declarerank1("trK_dKOD")
betU_dKOD    = ixp.declarerank2("betU_dKOD","nosym")
vetU_dKOD    = ixp.declarerank2("vetU_dKOD","nosym")
lambdaU_dKOD = ixp.declarerank2("lambdaU_dKOD","nosym")
aDD_dKOD = ixp.declarerank3("aDD_dKOD","sym01")
hDD_dKOD = ixp.declarerank3("hDD_dKOD","sym01")
for k in range(DIM):
    gaugerhs.alpha_rhs += diss_strength*alpha_dKOD[k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
    rhs.cf_rhs         += diss_strength*   cf_dKOD[k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
    rhs.trK_rhs        += diss_strength*  trK_dKOD[k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
    for i in range(DIM):
        if "2ndOrder" in ShiftCondition:
            gaugerhs.bet_rhsU[i] += diss_strength*   betU_dKOD[i][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
        gaugerhs.vet_rhsU[i]     += diss_strength*   vetU_dKOD[i][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
        rhs.lambda_rhsU[i]       += diss_strength*lambdaU_dKOD[i][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
        for j in range(DIM):
            rhs.a_rhsDD[i][j] += diss_strength*aDD_dKOD[i][j][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]
            rhs.h_rhsDD[i][j] += diss_strength*hDD_dKOD[i][j][k]*rfm.ReU[k] # ReU[k] = 1/scalefactor_orthog_funcform[k]

# We use betaU as our upwinding control vector:
Bq.BSSN_basic_tensors()
betaU = Bq.betaU

import BSSN.Enforce_Detgammabar_Constraint as EGC
enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammabar_Constraint_symb_expressions()

# Next compute Ricci tensor
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","False")
Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()

# Now that we are finished with all the rfm hatted
#           quantities in generic precomputed functional
#           form, let's restore them to their closed-
#           form expressions.
par.set_parval_from_str("reference_metric::enable_rfm_precompute","False") # Reset to False to disable rfm_precompute.
rfm.ref_metric__hatted_quantities()
end = time.time()
print("Finished BSSN symbolic expressions in "+str(end-start)+" seconds.")

def BSSN_RHSs():
    print("Generating C code for BSSN RHSs in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()

    BSSN_evol_rhss = [ \
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD00"),rhs=rhs.a_rhsDD[0][0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD01"),rhs=rhs.a_rhsDD[0][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD02"),rhs=rhs.a_rhsDD[0][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD11"),rhs=rhs.a_rhsDD[1][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD12"),rhs=rhs.a_rhsDD[1][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD22"),rhs=rhs.a_rhsDD[2][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","alpha"),rhs=gaugerhs.alpha_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU0"),rhs=gaugerhs.bet_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU1"),rhs=gaugerhs.bet_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU2"),rhs=gaugerhs.bet_rhsU[2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","cf"),   rhs=rhs.cf_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD00"),rhs=rhs.h_rhsDD[0][0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD01"),rhs=rhs.h_rhsDD[0][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD02"),rhs=rhs.h_rhsDD[0][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD11"),rhs=rhs.h_rhsDD[1][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD12"),rhs=rhs.h_rhsDD[1][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD22"),rhs=rhs.h_rhsDD[2][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU0"),rhs=rhs.lambda_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU1"),rhs=rhs.lambda_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU2"),rhs=rhs.lambda_rhsU[2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","trK"),  rhs=rhs.trK_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU0"),rhs=gaugerhs.vet_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU1"),rhs=gaugerhs.vet_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU2"),rhs=gaugerhs.vet_rhsU[2]) ]

    BSSN_RHSs_string = fin.FD_outputC("returnstring",BSSN_evol_rhss, params="outCverbose=False,SIMD_enable=True",
                                      upwindcontrolvec=betaU)

    with open(os.path.join(outdir,"BSSN_RHSs.h"), "w") as file:
        file.write(lp.loop(["i2","i1","i0"],["NGHOSTS","NGHOSTS","NGHOSTS"],
                           ["NGHOSTS+Nxx2","NGHOSTS+Nxx1","NGHOSTS+Nxx0"],
                           ["1","1","SIMD_width"],
                            ["#pragma omp parallel for",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read2.h\"",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read1.h\""],"",
                                         "#include \"rfm_files/rfm_struct__SIMD_inner_read0.h\"\n"+BSSN_RHSs_string))
    end = time.time()
    print("Finished BSSN_RHS C codegen in " + str(end - start) + " seconds.")
    
def Ricci():
    print("Generating C code for Ricci tensor in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    Ricci_string = fin.FD_outputC("returnstring",
                                  [lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD00"),rhs=Bq.RbarDD[0][0]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD01"),rhs=Bq.RbarDD[0][1]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD02"),rhs=Bq.RbarDD[0][2]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD11"),rhs=Bq.RbarDD[1][1]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD12"),rhs=Bq.RbarDD[1][2]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD22"),rhs=Bq.RbarDD[2][2])],
                                   params="outCverbose=False,SIMD_enable=True")
    with open(os.path.join(outdir,"BSSN_Ricci.h"), "w") as file:
        file.write(lp.loop(["i2","i1","i0"],["NGHOSTS","NGHOSTS","NGHOSTS"],
                           ["NGHOSTS+Nxx2","NGHOSTS+Nxx1","NGHOSTS+Nxx0"],
                           ["1","1","SIMD_width"],
                            ["#pragma omp parallel for",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read2.h\"",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read1.h\""],"",
                                         "#include \"rfm_files/rfm_struct__SIMD_inner_read0.h\"\n"+Ricci_string))
    end = time.time()
    print("Finished Ricci C codegen in " + str(end - start) + " seconds.")

Generating symbolic expressions for BSSN RHSs...
Finished BSSN symbolic expressions in 4.50829315186 seconds.


<a id='hammomconstraints'></a>

## Step 2.b: Hamiltonian & momentum constraints \[Back to [top](#toc)\]
$$\label{hammomconstraints}$$

Next output the C code for evaluating the Hamiltonian & momentum constraints [(**Tutorial**)](Tutorial-BSSN_constraints.ipynb). In the absence of numerical error, this constraint should evaluate to zero. However it does not due to numerical (typically truncation and roundoff) error. Therefore it is useful to measure the Hamiltonian & momentum constraint violation to gauge the accuracy of our simulation, and, ultimately determine whether errors are dominated by numerical finite differencing (truncation) error as expected.

In [3]:
# First register the Hamiltonian as a gridfunction.
H  = gri.register_gridfunctions("AUX","H")
MU = ixp.register_gridfunctions_for_single_rank1("AUX", "MU")

# Then define the Hamiltonian constraint and output the optimized C code.
import BSSN.BSSN_constraints as bssncon

def BSSNconstraints():
    bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)

    start = time.time()
    print("Generating optimized C code for Ham. & mom. constraints. May take a while, depending on CoordSystem.")
    Ham_mom_string = fin.FD_outputC("returnstring", 
                                    [lhrh(lhs=gri.gfaccess("aux_gfs", "H"),   rhs=bssncon.H),
                                     lhrh(lhs=gri.gfaccess("aux_gfs", "MU0"), rhs=bssncon.MU[0]),
                                     lhrh(lhs=gri.gfaccess("aux_gfs", "MU1"), rhs=bssncon.MU[1]),
                                     lhrh(lhs=gri.gfaccess("aux_gfs", "MU2"), rhs=bssncon.MU[2])],
                                    params="outCverbose=False")

    with open(os.path.join(outdir,"BSSN_constraints.h"), "w") as file:
        file.write("""
    void BSSN_constraints(const paramstruct *restrict params, REAL *restrict xx[3], 
                          REAL *restrict in_gfs, REAL *restrict aux_gfs) {
    #include "set_Cparameters.h"
    """)

        file.write(lp.loop(["i2","i1","i0"],["NGHOSTS","NGHOSTS","NGHOSTS"],
                           ["NGHOSTS+Nxx2","NGHOSTS+Nxx1","NGHOSTS+Nxx0"],
                           ["1","1","1"],
                           ["#pragma omp parallel for",
                            "    const REAL xx2 = xx[2][i2];",
                            "        const REAL xx1 = xx[1][i1];"], "",
                               "const REAL xx0 = xx[0][i0];\n" + Ham_mom_string))
        file.write("}\n")
    end = time.time()
    print("Finished Hamiltonian & momentum constraint C codegen in " + str(end - start) + " seconds.")

<a id='gamconstraint'></a>

## Step 2.c: Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (in Cartesian coordinates, $\det{\hat{\gamma}_{ij}}=1$) \[Back to [top](#toc)\]
$$\label{gamconstraint}$$

Then enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (Eq. 53 of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)), as [documented in the corresponding NRPy+ tutorial module](Tutorial-BSSN-Enforcing_Determinant_gammabar_equals_gammahat_Constraint.ipynb)

Applying curvilinear boundary conditions should affect the initial data at the outer boundary, and will in general cause the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint to be violated there. Thus after we apply these boundary conditions, we must always call the routine for enforcing the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint:

In [4]:
def gammadet():
    start = time.time()
    print("Generating optimized C code for gamma constraint. May take a while, depending on CoordSystem.")
    enforce_gammadet_string = fin.FD_outputC("returnstring", enforce_detg_constraint_symb_expressions,
                                             params="outCverbose=False,preindent=0,includebraces=False")

    with open(os.path.join(outdir,"enforce_detgammabar_constraint.h"), "w") as file:
        indent = "   "
        file.write("""
    void enforce_detgammabar_constraint(rfm_struct *restrict rfmstruct,
                                        const paramstruct *restrict params, REAL *restrict in_gfs) {
    #include "set_Cparameters.h"
    """)
        file.write(lp.loop(["i2","i1","i0"],["0", "0", "0"],
                           ["Nxx_plus_2NGHOSTS2", "Nxx_plus_2NGHOSTS1", "Nxx_plus_2NGHOSTS0"],
                           ["1","1","1"],
                            ["#pragma omp parallel for",
                                         "#include \"rfm_files/rfm_struct__read2.h\"",
                                         "#include \"rfm_files/rfm_struct__read1.h\""],"",
                                         "#include \"rfm_files/rfm_struct__read0.h\"\n"+enforce_gammadet_string))
        file.write("}\n")
    end = time.time()
    print("Finished gamma constraint C codegen in " + str(end - start) + " seconds.")

<a id='parallel_codegen'></a>

## Step 2.d: Generate all C codes in parallel \[Back to [top](#toc)\]
$$\label{parallel_codegen}$$


In [5]:
import multiprocessing

if __name__ == '__main__':
    RHS = multiprocessing.Process(target=BSSN_RHSs)
    Ric = multiprocessing.Process(target=Ricci)
    con = multiprocessing.Process(target=BSSNconstraints)
    gam = multiprocessing.Process(target=gammadet)
    
    RHS.start()
    Ric.start()
    con.start()
    gam.start()
    
    RHS.join()
    Ric.join()
    con.join()
    gam.join()

Generating C code for BSSN RHSs in Cartesian coordinates.
Generating C code for Ricci tensor in Cartesian coordinates.
Generating optimized C code for gamma constraint. May take a while, depending on CoordSystem.
Finished gamma constraint C codegen in 0.411771059036 seconds.
Generating optimized C code for Ham. & mom. constraints. May take a while, depending on CoordSystem.
Finished Hamiltonian & momentum constraint C codegen in 10.4662559032 seconds.
Finished BSSN_RHS C codegen in 74.3176720142 seconds.
Finished Ricci C codegen in 79.0024738312 seconds.


<a id='cclfiles'></a>

# Step 3: ETK `ccl` file generation \[Back to [top](#toc)\]
$$\label{cclfiles}$$

<a id='paramccl'></a>

## Step 3.a: `param.ccl` \[Back to [top](#toc)\]
$$\label{paramccl}$$

All parameters necessary for the computation of the BSSN right-hand side (RHS) expressions are registered within NRPy+; we use this information to automatically generate `param.ccl`. NRPy+ also specifies default values for each parameter. 

More information on `param.ccl` syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [9]:
with open(os.path.join(outrootdir,"param.ccl"), "w") as file:
    file.write("""
# This param.ccl file was automatically generated by NRPy+. 
#   You are advised against modifying it directly.

shares: ADMBase
EXTENDS CCTK_KEYWORD evolution_method "evolution_method"
{
  "BaikalETK" :: ""
} 

EXTENDS CCTK_KEYWORD lapse_evolution_method "lapse_evolution_method"
{
  "BaikalETK" :: ""
} 

EXTENDS CCTK_KEYWORD shift_evolution_method "shift_evolution_method"
{
  "BaikalETK" :: ""
} 

EXTENDS CCTK_KEYWORD dtshift_evolution_method "dtshift_evolution_method"
{
  "BaikalETK" :: ""
} 

restricted:
""")
    paramccl_str = ""
    for i in range(len(par.glb_Cparams_list)):
        singleparstring = ""
        keep_param = True # We'll not set some parameters in param.ccl; 
                          #   e.g., those that should be #define'd like M_PI.
        
        # Separate thorns within the ETK take care of grid/coordinate parameters;
        #   thus we ignore NRPy+ grid/coordinate parameters:
        if par.glb_Cparams_list[i].module == "grid" or par.glb_Cparams_list[i].module == "reference_metric":
            keep_param = False
        
        partype = par.glb_Cparams_list[i].type
        if partype == "bool":
            singleparstring += "BOOLEAN "
        elif partype == "REAL":
            if par.glb_Cparams_list[i].defaultval != 1e300: # 1e300 is a magic value indicating that the C parameter should be mutable
                singleparstring += "REAL "
            else:
                keep_param = False
        elif partype == "int":
            singleparstring += "INT "
        elif partype == "#define":
            keep_param = False
        elif partype == "char":
            # FIXME
            print("Error: parameter "+par.glb_Cparams_list[i].module+"::"+par.glb_Cparams_list[i].parname+
                  " has unsupported type: \""+ par.glb_Cparams_list[i].type + "\"")
            sys.exit(1)
        else:
            print("Error: parameter "+par.glb_Cparams_list[i].module+"::"+par.glb_Cparams_list[i].parname+
                  " has unsupported type: \""+ par.glb_Cparams_list[i].type + "\"")
            sys.exit(1)
#         print(partype,keep_param)
        if keep_param:
            singleparstring += par.glb_Cparams_list[i].parname + " \""+ par.glb_Cparams_list[i].parname +" (see NRPy+ for parameter definition)\"\n"
            singleparstring += "{\n"
            if partype != "bool":
                singleparstring += " *:* :: \"All values accepted. NRPy+ does not restrict the allowed ranges of parameters yet.\"\n"
            singleparstring += "} "+str(par.glb_Cparams_list[i].defaultval)+"\n\n"
            
            paramccl_str += singleparstring
    file.write(paramccl_str)

<a id='latex_pdf_output'></a>

# Step N: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-BaikalETK.pdf](Tutorial-BaikalETK.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [7]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-BaikalETK.ipynb
!pdflatex -interaction=batchmode Tutorial-BaikalETK.tex
!pdflatex -interaction=batchmode Tutorial-BaikalETK.tex
!pdflatex -interaction=batchmode Tutorial-BaikalETK.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-BaikalETK.ipynb to latex
[NbConvertApp] Writing 86604 bytes to Tutorial-BaikalETK.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
